In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
restaurant = pd.read_csv('zomato.csv')

restaurant.shape
restaurant.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [2]:
restaurant = restaurant[['url', 'address', 'name', 'online_order', 'rate', 'votes', 'phone', 'location', 'rest_type', 'dish_liked', 'cuisines', 'approx_cost(for two people)', 'listed_in(type)', 'listed_in(city)', 'reviews_list']]
restaurant.head(1)

,url,address,name,online_order,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),listed_in(type),listed_in(city),reviews_list
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,Buffet,Banashankari,"[('Rated 4.0', 'RATED\n A beautiful place to ..."


In [3]:
restaurant.dropna(subset=['reviews_list'], inplace=True)
restaurant['phone'].fillna('Not Available', inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_15928\3560435439.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  restaurant['phone'].fillna('Not Available', inplace=True)


In [4]:
restaurant.isnull().sum()

url                                0
address                            0
name                               0
online_order                       0
rate                            7775
votes                              0
phone                              0
location                          21
rest_type                        227
dish_liked                     28078
cuisines                          45
approx_cost(for two people)      346
listed_in(type)                    0
listed_in(city)                    0
reviews_list                       0
dtype: int64

In [5]:
restaurant['rate'] = restaurant['rate'].str.split('/').str[0]
restaurant['rate'] = pd.to_numeric(restaurant['rate'], errors='coerce')
restaurant.dropna(subset=['rest_type'], inplace=True)

In [6]:
restaurant.head(1)

,url,address,name,online_order,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),listed_in(type),listed_in(city),reviews_list
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,4.1,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,Buffet,Banashankari,"[('Rated 4.0', 'RATED\n A beautiful place to ..."


In [7]:
restaurant.isnull().sum()

url                                0
address                            0
name                               0
online_order                       0
rate                            9974
votes                              0
phone                              0
location                           0
rest_type                          0
dish_liked                     27921
cuisines                          24
approx_cost(for two people)      323
listed_in(type)                    0
listed_in(city)                    0
reviews_list                       0
dtype: int64

In [8]:
restaurant.shape

(51490, 15)

In [9]:
restaurant = restaurant.drop(columns=['reviews_list'])
unique_restaurants_df = restaurant.drop_duplicates(subset=['name', 'address'])

In [10]:
restaurant=unique_restaurants_df

In [11]:
restaurant.head(1)

,url,address,name,online_order,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,4.1,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,Buffet,Banashankari


In [12]:
restaurant.dropna(subset=['dish_liked'], inplace=True)

In [13]:
restaurant.to_csv('cleaned_zomato.csv', index=False)

In [14]:
restaurant.shape

(4592, 14)

In [15]:
restaurant['dish_liked'] = restaurant['dish_liked'].apply(lambda x: ','.join(i.replace(" ", "") for i in x.split(',')))
restaurant['cuisines'] = restaurant['cuisines'].apply(lambda x: ','.join(i.replace(" ", "") for i in x.split(',')))
restaurant['rest_type'] = restaurant['rest_type'].apply(lambda x: ','.join(i.replace(" ", "") for i in x.split(',')))
restaurant['rest_type'] = restaurant['rest_type'].apply(lambda x: x.split())
restaurant['dish_liked'] = restaurant['dish_liked'].apply(lambda x: x.split())
restaurant['cuisines'] = restaurant['cuisines'].apply(lambda x: x.split())
restaurant['listed_in(type)'] = restaurant['listed_in(type)'].apply(lambda x: x.split())

In [16]:
restaurant.head()

,url,address,name,online_order,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,4.1,775,080 42297555\r\n+91 9743772233,Banashankari,[CasualDining],"[Pasta,LunchBuffet,MasalaPapad,PaneerLajawab,T...","[NorthIndian,Mughlai,Chinese]",800,[Buffet],Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,4.1,787,080 41714161,Banashankari,[CasualDining],"[Momos,LunchBuffet,ChocolateNirvana,ThaiGreenC...","[Chinese,NorthIndian,Thai]",800,[Buffet],Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,3.8,918,+91 9663487993,Banashankari,"[Cafe,CasualDining]","[Churros,Cannelloni,MinestroneSoup,HotChocolat...","[Cafe,Mexican,Italian]",800,[Buffet],Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,3.7,88,+91 9620009302,Banashankari,[QuickBites],[MasalaDosa],"[SouthIndian,NorthIndian]",300,[Buffet],Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,3.8,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,[CasualDining],"[Panipuri,GolGappe]","[NorthIndian,Rajasthani]",600,[Buffet],Banashankari


In [17]:
for index, row in restaurant.iterrows():
    if row['online_order'] == 'No':
        restaurant.at[index, 'url'] = "restaurant doesn't deliver"
restaurant.rename(columns={'url': 'delivery_url'}, inplace=True)

# Creating Google Maps link
def create_google_maps_url(address):
    base_url = "https://www.google.com/maps/search/?api=1&query="
    return base_url + address.replace(' ', '+')
restaurant['google_maps_link'] = restaurant['address'].apply(create_google_maps_url)


In [18]:
restaurant.head()

,delivery_url,address,name,online_order,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),listed_in(type),listed_in(city),google_maps_link
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,4.1,775,080 42297555\r\n+91 9743772233,Banashankari,[CasualDining],"[Pasta,LunchBuffet,MasalaPapad,PaneerLajawab,T...","[NorthIndian,Mughlai,Chinese]",800,[Buffet],Banashankari,https://www.google.com/maps/search/?api=1&quer...
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,4.1,787,080 41714161,Banashankari,[CasualDining],"[Momos,LunchBuffet,ChocolateNirvana,ThaiGreenC...","[Chinese,NorthIndian,Thai]",800,[Buffet],Banashankari,https://www.google.com/maps/search/?api=1&quer...
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,3.8,918,+91 9663487993,Banashankari,"[Cafe,CasualDining]","[Churros,Cannelloni,MinestroneSoup,HotChocolat...","[Cafe,Mexican,Italian]",800,[Buffet],Banashankari,https://www.google.com/maps/search/?api=1&quer...
3,restaurant doesn't deliver,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,3.7,88,+91 9620009302,Banashankari,[QuickBites],[MasalaDosa],"[SouthIndian,NorthIndian]",300,[Buffet],Banashankari,https://www.google.com/maps/search/?api=1&quer...
4,restaurant doesn't deliver,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,3.8,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,[CasualDining],"[Panipuri,GolGappe]","[NorthIndian,Rajasthani]",600,[Buffet],Banashankari,https://www.google.com/maps/search/?api=1&quer...


In [19]:
restaurant['approx_cost(for two people)'].fillna('Not Available', inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_15928\1838381742.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  restaurant['approx_cost(for two people)'].fillna('Not Available', inplace=True)


In [20]:
restaurant.to_csv('cleaned_zomato.csv', index=False)

In [21]:
restaurant.isnull().sum()

delivery_url                    0
address                         0
name                            0
online_order                    0
rate                           31
votes                           0
phone                           0
location                        0
rest_type                       0
dish_liked                      0
cuisines                        0
approx_cost(for two people)     0
listed_in(type)                 0
listed_in(city)                 0
google_maps_link                0
dtype: int64

In [22]:
restaurant.head(1)

,delivery_url,address,name,online_order,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),listed_in(type),listed_in(city),google_maps_link
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,4.1,775,080 42297555\r\n+91 9743772233,Banashankari,[CasualDining],"[Pasta,LunchBuffet,MasalaPapad,PaneerLajawab,T...","[NorthIndian,Mughlai,Chinese]",800,[Buffet],Banashankari,https://www.google.com/maps/search/?api=1&quer...


In [23]:
restaurant.iloc[0].cuisines

['NorthIndian,Mughlai,Chinese']

In [24]:
fitered = restaurant

In [25]:
def split_string_to_list(text):
    # Remove any leading/trailing whitespace and split by comma
    return [item.strip() for item in text[0].split(',')]

# Apply the function to 'dish_liked' column
restaurant['rest_type'] = restaurant['rest_type'].apply(split_string_to_list)
restaurant['dish_liked'] = restaurant['dish_liked'].apply(split_string_to_list)
restaurant['cuisines'] = restaurant['cuisines'].apply(split_string_to_list)
restaurant['listed_in(type)'] = restaurant['listed_in(type)'].apply(split_string_to_list)

In [26]:
restaurant.isnull().sum()

delivery_url                    0
address                         0
name                            0
online_order                    0
rate                           31
votes                           0
phone                           0
location                        0
rest_type                       0
dish_liked                      0
cuisines                        0
approx_cost(for two people)     0
listed_in(type)                 0
listed_in(city)                 0
google_maps_link                0
dtype: int64

In [27]:
restaurant.head(1)

,delivery_url,address,name,online_order,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),listed_in(type),listed_in(city),google_maps_link
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,4.1,775,080 42297555\r\n+91 9743772233,Banashankari,[CasualDining],"[Pasta, LunchBuffet, MasalaPapad, PaneerLajawa...","[NorthIndian, Mughlai, Chinese]",800,[Buffet],Banashankari,https://www.google.com/maps/search/?api=1&quer...


In [28]:
restaurant.iloc[0].dish_liked

['Pasta',
 'LunchBuffet',
 'MasalaPapad',
 'PaneerLajawab',
 'TomatoShorba',
 'DumBiryani',
 'SweetCornSoup']

In [29]:
restaurant.head()

,delivery_url,address,name,online_order,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),listed_in(type),listed_in(city),google_maps_link
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,4.1,775,080 42297555\r\n+91 9743772233,Banashankari,[CasualDining],"[Pasta, LunchBuffet, MasalaPapad, PaneerLajawa...","[NorthIndian, Mughlai, Chinese]",800,[Buffet],Banashankari,https://www.google.com/maps/search/?api=1&quer...
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,4.1,787,080 41714161,Banashankari,[CasualDining],"[Momos, LunchBuffet, ChocolateNirvana, ThaiGre...","[Chinese, NorthIndian, Thai]",800,[Buffet],Banashankari,https://www.google.com/maps/search/?api=1&quer...
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,3.8,918,+91 9663487993,Banashankari,"[Cafe, CasualDining]","[Churros, Cannelloni, MinestroneSoup, HotChoco...","[Cafe, Mexican, Italian]",800,[Buffet],Banashankari,https://www.google.com/maps/search/?api=1&quer...
3,restaurant doesn't deliver,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,3.7,88,+91 9620009302,Banashankari,[QuickBites],[MasalaDosa],"[SouthIndian, NorthIndian]",300,[Buffet],Banashankari,https://www.google.com/maps/search/?api=1&quer...
4,restaurant doesn't deliver,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,3.8,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,[CasualDining],"[Panipuri, GolGappe]","[NorthIndian, Rajasthani]",600,[Buffet],Banashankari,https://www.google.com/maps/search/?api=1&quer...


In [30]:
restaurant['cuisines']

0                    [NorthIndian, Mughlai, Chinese]
1                       [Chinese, NorthIndian, Thai]
2                           [Cafe, Mexican, Italian]
3                         [SouthIndian, NorthIndian]
4                          [NorthIndian, Rajasthani]
                            ...                     
51607                                   [FingerFood]
51611                               [Chinese, Momos]
51620                         [Thai, Chinese, Momos]
51628              [NorthIndian, Continental, Asian]
51677    [Andhra, SouthIndian, Chinese, NorthIndian]
Name: cuisines, Length: 4592, dtype: object

In [31]:
restaurant = restaurant[restaurant['dish_liked'].apply(len) > 2]

In [32]:
restaurant.shape

(3689, 15)

In [33]:
restaurant.to_csv('cleaned_zomato.csv', index=False)

In [34]:
restaurant.head(5)

,delivery_url,address,name,online_order,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),listed_in(type),listed_in(city),google_maps_link
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,4.1,775,080 42297555\r\n+91 9743772233,Banashankari,[CasualDining],"[Pasta, LunchBuffet, MasalaPapad, PaneerLajawa...","[NorthIndian, Mughlai, Chinese]",800,[Buffet],Banashankari,https://www.google.com/maps/search/?api=1&quer...
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,4.1,787,080 41714161,Banashankari,[CasualDining],"[Momos, LunchBuffet, ChocolateNirvana, ThaiGre...","[Chinese, NorthIndian, Thai]",800,[Buffet],Banashankari,https://www.google.com/maps/search/?api=1&quer...
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,3.8,918,+91 9663487993,Banashankari,"[Cafe, CasualDining]","[Churros, Cannelloni, MinestroneSoup, HotChoco...","[Cafe, Mexican, Italian]",800,[Buffet],Banashankari,https://www.google.com/maps/search/?api=1&quer...
5,https://www.zomato.com/bangalore/timepass-dinn...,"37, 5-1, 4th Floor, Bosco Court, Gandhi Bazaar...",Timepass Dinner,Yes,3.8,286,+91 9980040002\r\n+91 9980063005,Basavanagudi,[CasualDining],"[OnionRings, Pasta, KadhaiPaneer, Salads, Sala...",[NorthIndian],600,[Buffet],Banashankari,https://www.google.com/maps/search/?api=1&quer...
7,https://www.zomato.com/bangalore/onesta-banash...,"2469, 3rd Floor, 24th Cross, Opposite BDA Comp...",Onesta,Yes,4.6,2556,080 48653961\r\n080 48655715,Banashankari,"[CasualDining, Cafe]","[FarmhousePizza, ChocolateBanana, VirginMojito...","[Pizza, Cafe, Italian]",600,[Cafes],Banashankari,https://www.google.com/maps/search/?api=1&quer...


In [35]:
restaurant['tags'] = restaurant['rest_type'] + restaurant['dish_liked'] + restaurant['cuisines'] + restaurant['listed_in(type)']


C:\Users\user\AppData\Local\Temp\ipykernel_15928\4239165207.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  restaurant['tags'] = restaurant['rest_type'] + restaurant['dish_liked'] + restaurant['cuisines'] + restaurant['listed_in(type)']


In [36]:
restaurant.to_csv('cleaned_zomato.csv', index=False)

In [37]:
restaurant['tags']

0        [CasualDining, Pasta, LunchBuffet, MasalaPapad...
1        [CasualDining, Momos, LunchBuffet, ChocolateNi...
2        [Cafe, CasualDining, Churros, Cannelloni, Mine...
5        [CasualDining, OnionRings, Pasta, KadhaiPaneer...
7        [CasualDining, Cafe, FarmhousePizza, Chocolate...
                               ...                        
51515    [FoodCourt, CasualDining, VegetableBiryani, No...
51606    [CasualDining, Pizza, Pasta, Salads, PotatoMas...
51607    [Bar, Cocktails, Pizza, Buttermilk, FingerFood...
51620    [FoodCourt, Noodles, ChickenNoodle, Momos, Ame...
51628    [CasualDining, ChickenQuesadilla, Naan, Breakf...
Name: tags, Length: 3689, dtype: object

In [38]:
restaurant['tags'].apply(lambda x:" ".join(x))

0        CasualDining Pasta LunchBuffet MasalaPapad Pan...
1        CasualDining Momos LunchBuffet ChocolateNirvan...
2        Cafe CasualDining Churros Cannelloni Minestron...
5        CasualDining OnionRings Pasta KadhaiPaneer Sal...
7        CasualDining Cafe FarmhousePizza ChocolateBana...
                               ...                        
51515    FoodCourt CasualDining VegetableBiryani NonVeg...
51606    CasualDining Pizza Pasta Salads PotatoMash Bru...
51607    Bar Cocktails Pizza Buttermilk FingerFood Dine...
51620    FoodCourt Noodles ChickenNoodle Momos American...
51628    CasualDining ChickenQuesadilla Naan BreakfastB...
Name: tags, Length: 3689, dtype: object

In [39]:
restaurant['tags'] = restaurant['tags'].apply(lambda x:" ".join(x))

C:\Users\user\AppData\Local\Temp\ipykernel_15928\1082890595.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  restaurant['tags'] = restaurant['tags'].apply(lambda x:" ".join(x))


In [40]:
restaurant['tags'][0]

'CasualDining Pasta LunchBuffet MasalaPapad PaneerLajawab TomatoShorba DumBiryani SweetCornSoup NorthIndian Mughlai Chinese Buffet'

In [41]:
restaurant['tags'] = restaurant['tags'].apply(lambda x:x.lower())

C:\Users\user\AppData\Local\Temp\ipykernel_15928\2999407499.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  restaurant['tags'] = restaurant['tags'].apply(lambda x:x.lower())


In [42]:
restaurant.to_csv('cleaned_zomato.csv', index=False)

In [43]:
# Assuming `restaurant` is your DataFrame
#restaurant.rename(columns={'delivery url': 'delivery_url'}, inplace=True)


In [44]:
def recommend(restaurant_name):
    target_tags = set(restaurant[restaurant['name'] == restaurant_name]['tags'].iloc[0].split())
    similar_restaurants = []

    for idx, row in restaurant.iterrows():
        if row['name'] != restaurant_name:
            similarity_score = len(target_tags.intersection(set(row['tags'].split()))) / len(target_tags.union(set(row['tags'].split())))
            similar_restaurants.append((row['name'], similarity_score, row['google_maps_link'], row['delivery_url']))

    similar_restaurants = sorted(similar_restaurants, key=lambda x: x[1], reverse=True)[:5]

    print(f"Input Restaurant: {restaurant_name}")
    print(f"Google Maps Link: {restaurant[restaurant['name'] == restaurant_name]['google_maps_link'].iloc[0]}")
    print(f"Delivery URL: {restaurant[restaurant['name'] == restaurant_name]['delivery_url'].iloc[0]}")
    
    print("\nRecommended Restaurants:")
    
    for rest in similar_restaurants:
        print(f"Name: {rest[0]}, Google Maps Link: {rest[2]}\nDelivery URL: {rest[3]}")
        print()

# Example usage: Recommend restaurants similar to 'Onesta'
recommend('Onesta')


Input Restaurant: Onesta
Google Maps Link: https://www.google.com/maps/search/?api=1&query=2469,+3rd+Floor,+24th+Cross,+Opposite+BDA+Complex,+2nd+Stage,+Banashankari,+Bangalore
Delivery URL: https://www.zomato.com/bangalore/onesta-banashankari?context=eyJzZSI6eyJlIjpbMTg1NjU2NzksIjE4NTkzMDQ0IiwiMTgxNDg5NDEiLCIxODQ3MDc3MSIsIjE4NzE2MDYxIiwiMTg2MzI5NzciLCIxODIwMjk5NyIsIjU5MDkwIiwiMTg1NTkyMTEiLCIxODMyMzYzOSIsIjE4NTc0Mjc3Il0sInQiOiJDYWZcdTAwZTlzIGFuZCBEZWxpcyBpbiBCYW5hc2hhbmthcmkifX0=

Recommended Restaurants:
Name: Enchanted Garden, Google Maps Link: https://www.google.com/maps/search/?api=1&query=Bungalow+1,+Bright+Farm,+Thubarahalli,+Varthur+Road,+Whitefield,+Bangalore
Delivery URL: https://www.zomato.com/bangalore/enchanted-garden-whitefield-bangalore?context=eyJzZSI6eyJlIjpbIjE4ODExMDQ0IiwiMTgzNzk2NjEiLCIxODI3NTAwOCIsIjE4MzkxMDIwIiwxODc5MTQ2MiwiNTY3NjciLCIxODY3MDAzMSIsIjE4NzYyNjk4IiwiMTg1NTg2MDciLCIxODg5MjM3MiIsIjE4MjMxNDE0IiwiMTg1MDIxMTgiLCIxODY3MzU5MSIsIjE4Nzk4NjUwIiwiNTQxNTUiLCIxODY

In [45]:
restaurant['address'][0]

'942, 21st Main Road, 2nd Stage, Banashankari, Bangalore'

In [54]:
import pickle

In [55]:
pickle.dump(restaurant,open('restaurant_list.pkl','wb'))

In [56]:
restaurant.head()

,delivery_url,address,name,online_order,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),listed_in(type),listed_in(city),google_maps_link,tags
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,4.1,775,080 42297555\r\n+91 9743772233,Banashankari,[CasualDining],"[Pasta, LunchBuffet, MasalaPapad, PaneerLajawa...","[NorthIndian, Mughlai, Chinese]",800,[Buffet],Banashankari,https://www.google.com/maps/search/?api=1&quer...,casualdining pasta lunchbuffet masalapapad pan...
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,4.1,787,080 41714161,Banashankari,[CasualDining],"[Momos, LunchBuffet, ChocolateNirvana, ThaiGre...","[Chinese, NorthIndian, Thai]",800,[Buffet],Banashankari,https://www.google.com/maps/search/?api=1&quer...,casualdining momos lunchbuffet chocolatenirvan...
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,3.8,918,+91 9663487993,Banashankari,"[Cafe, CasualDining]","[Churros, Cannelloni, MinestroneSoup, HotChoco...","[Cafe, Mexican, Italian]",800,[Buffet],Banashankari,https://www.google.com/maps/search/?api=1&quer...,cafe casualdining churros cannelloni minestron...
5,https://www.zomato.com/bangalore/timepass-dinn...,"37, 5-1, 4th Floor, Bosco Court, Gandhi Bazaar...",Timepass Dinner,Yes,3.8,286,+91 9980040002\r\n+91 9980063005,Basavanagudi,[CasualDining],"[OnionRings, Pasta, KadhaiPaneer, Salads, Sala...",[NorthIndian],600,[Buffet],Banashankari,https://www.google.com/maps/search/?api=1&quer...,casualdining onionrings pasta kadhaipaneer sal...
7,https://www.zomato.com/bangalore/onesta-banash...,"2469, 3rd Floor, 24th Cross, Opposite BDA Comp...",Onesta,Yes,4.6,2556,080 48653961\r\n080 48655715,Banashankari,"[CasualDining, Cafe]","[FarmhousePizza, ChocolateBanana, VirginMojito...","[Pizza, Cafe, Italian]",600,[Cafes],Banashankari,https://www.google.com/maps/search/?api=1&quer...,casualdining cafe farmhousepizza chocolatebana...
